# Réseaux neuronaux récurrents

Dans le module précédent, nous avons abordé les représentations sémantiques riches du texte. L'architecture que nous avons utilisée capture le sens global des mots dans une phrase, mais elle ne prend pas en compte l'**ordre** des mots, car l'opération d'agrégation qui suit les embeddings élimine cette information du texte original. Étant donné que ces modèles ne peuvent pas représenter l'ordre des mots, ils ne peuvent pas résoudre des tâches plus complexes ou ambiguës comme la génération de texte ou la réponse à des questions.

Pour capturer le sens d'une séquence de texte, nous utiliserons une architecture de réseau neuronal appelée **réseau neuronal récurrent**, ou RNN. Lorsqu'on utilise un RNN, on fait passer notre phrase à travers le réseau un jeton à la fois, et le réseau produit un certain **état**, que l'on transmet ensuite au réseau avec le jeton suivant.

![Image montrant un exemple de génération par réseau neuronal récurrent.](../../../../../lessons/5-NLP/16-RNN/images/rnn.png)

Étant donné la séquence d'entrée de jetons $X_0,\dots,X_n$, le RNN crée une séquence de blocs de réseau neuronal et entraîne cette séquence de bout en bout en utilisant la rétropropagation. Chaque bloc de réseau prend une paire $(X_i,S_i)$ en entrée et produit $S_{i+1}$ en résultat. L'état final $S_n$ ou la sortie $Y_n$ est ensuite transmis à un classificateur linéaire pour produire le résultat. Tous les blocs de réseau partagent les mêmes poids et sont entraînés de bout en bout en une seule passe de rétropropagation.

> La figure ci-dessus montre un réseau neuronal récurrent sous forme déroulée (à gauche) et sous une représentation récurrente plus compacte (à droite). Il est important de comprendre que toutes les cellules RNN partagent les mêmes **poids partageables**.

Comme les vecteurs d'état $S_0,\dots,S_n$ sont transmis à travers le réseau, le RNN est capable d'apprendre les dépendances séquentielles entre les mots. Par exemple, lorsque le mot *pas* apparaît quelque part dans la séquence, il peut apprendre à négativer certains éléments dans le vecteur d'état.

À l'intérieur, chaque cellule RNN contient deux matrices de poids : $W_H$ et $W_I$, ainsi qu'un biais $b$. À chaque étape du RNN, étant donné l'entrée $X_i$ et l'état d'entrée $S_i$, l'état de sortie est calculé comme $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, où $f$ est une fonction d'activation (souvent $\tanh$).

> Pour des problèmes comme la génération de texte (que nous aborderons dans la prochaine unité) ou la traduction automatique, nous souhaitons également obtenir une valeur de sortie à chaque étape du RNN. Dans ce cas, il y a une autre matrice $W_O$, et la sortie est calculée comme $Y_i=f(W_O\times S_i+b_O)$.

Voyons comment les réseaux neuronaux récurrents peuvent nous aider à classifier notre ensemble de données de nouvelles.

> Pour l'environnement sandbox, nous devons exécuter la cellule suivante pour nous assurer que la bibliothèque requise est installée et que les données sont préchargées. Si vous travaillez en local, vous pouvez ignorer la cellule suivante.


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

Lors de l'entraînement de modèles de grande taille, l'allocation de mémoire GPU peut poser problème. Nous pourrions également avoir besoin d'expérimenter avec différentes tailles de minibatch, afin que les données tiennent dans la mémoire GPU tout en garantissant un entraînement suffisamment rapide. Si vous exécutez ce code sur votre propre machine équipée d'un GPU, vous pouvez essayer d'ajuster la taille des minibatchs pour accélérer l'entraînement.

> **Note** : Certaines versions des pilotes NVidia sont connues pour ne pas libérer la mémoire après l'entraînement du modèle. Nous exécutons plusieurs exemples dans ce notebook, ce qui pourrait entraîner une saturation de la mémoire dans certains cas, en particulier si vous réalisez vos propres expériences dans le même notebook. Si vous rencontrez des erreurs étranges au moment de commencer l'entraînement du modèle, il peut être utile de redémarrer le noyau du notebook.


In [3]:
batch_size = 16
embed_size = 64

## Classificateur RNN simple

Dans le cas d'un RNN simple, chaque unité récurrente est un réseau linéaire simple, qui prend en entrée un vecteur d'entrée et un vecteur d'état, et produit un nouveau vecteur d'état. Dans Keras, cela peut être représenté par la couche `SimpleRNN`.

Bien que nous puissions transmettre directement des tokens encodés en one-hot à la couche RNN, ce n'est pas une bonne idée en raison de leur haute dimensionnalité. Par conséquent, nous utiliserons une couche d'embedding pour réduire la dimensionnalité des vecteurs de mots, suivie d'une couche RNN, et enfin d'un classificateur `Dense`.

> **Note** : Dans les cas où la dimensionnalité n'est pas si élevée, par exemple lors de l'utilisation de la tokenisation au niveau des caractères, il peut être pertinent de transmettre directement les tokens encodés en one-hot dans la cellule RNN.


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **Note:** Nous utilisons ici une couche d'embedding non entraînée pour simplifier, mais pour obtenir de meilleurs résultats, nous pouvons utiliser une couche d'embedding préentraînée avec Word2Vec, comme décrit dans l'unité précédente. Ce serait un bon exercice pour vous d'adapter ce code afin de fonctionner avec des embeddings préentraînés.

Passons maintenant à l'entraînement de notre RNN. Les RNNs sont généralement assez difficiles à entraîner, car une fois que les cellules RNN sont déroulées sur la longueur de la séquence, le nombre de couches impliquées dans la rétropropagation devient très important. Par conséquent, nous devons sélectionner un taux d'apprentissage plus faible et entraîner le réseau sur un ensemble de données plus large pour obtenir de bons résultats. Cela peut prendre beaucoup de temps, donc l'utilisation d'un GPU est préférable.

Pour accélérer les choses, nous allons entraîner le modèle RNN uniquement sur les titres des actualités, en omettant la description. Vous pouvez essayer d'entraîner avec la description et voir si vous parvenez à faire fonctionner le modèle.


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **Note** que la précision est probablement plus faible ici, car nous nous entraînons uniquement sur les titres des actualités.


## Revoir les séquences de variables

Rappelez-vous que la couche `TextVectorization` ajoutera automatiquement des tokens de remplissage aux séquences de longueur variable dans un minibatch. Il s'avère que ces tokens participent également à l'entraînement, ce qui peut compliquer la convergence du modèle.

Il existe plusieurs approches pour minimiser la quantité de remplissage. L'une d'elles consiste à réorganiser le dataset par longueur de séquence et à regrouper toutes les séquences par taille. Cela peut être réalisé en utilisant la fonction `tf.data.experimental.bucket_by_sequence_length` (voir [documentation](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length)).

Une autre approche consiste à utiliser **le masquage**. Dans Keras, certaines couches prennent en charge des entrées supplémentaires qui indiquent quels tokens doivent être pris en compte lors de l'entraînement. Pour intégrer le masquage dans notre modèle, nous pouvons soit inclure une couche `Masking` séparée ([docs](https://keras.io/api/layers/core_layers/masking/)), soit spécifier le paramètre `mask_zero=True` dans notre couche `Embedding`.

> **Note** : Cet entraînement prendra environ 5 minutes pour compléter une époque sur l'ensemble du dataset. N'hésitez pas à interrompre l'entraînement à tout moment si vous manquez de patience. Ce que vous pouvez également faire, c'est limiter la quantité de données utilisées pour l'entraînement, en ajoutant une clause `.take(...)` après les datasets `ds_train` et `ds_test`.


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


Maintenant que nous utilisons le masquage, nous pouvons entraîner le modèle sur l'ensemble du jeu de données des titres et descriptions.

> **Note** : Avez-vous remarqué que nous avons utilisé un vectoriseur entraîné sur les titres des actualités, et non sur l'intégralité du corps de l'article ? Cela peut potentiellement entraîner l'ignorance de certains tokens, il serait donc préférable de réentraîner le vectoriseur. Cependant, l'impact pourrait être très minime, donc nous continuerons à utiliser le vectoriseur pré-entraîné précédent pour des raisons de simplicité.


## LSTM : Mémoire à long et court terme

L'un des principaux problèmes des RNN est le phénomène de **gradients évanescents**. Les RNN peuvent être assez longs et peuvent avoir du mal à propager les gradients jusqu'à la première couche du réseau lors de la rétropropagation. Lorsque cela se produit, le réseau ne peut pas apprendre les relations entre des tokens éloignés. Une façon d'éviter ce problème est d'introduire une **gestion explicite de l'état** en utilisant des **portes**. Les deux architectures les plus courantes qui introduisent des portes sont la **mémoire à long et court terme** (LSTM) et l'**unité de relais à portes** (GRU). Nous allons nous concentrer ici sur les LSTM.

![Image montrant un exemple de cellule de mémoire à long et court terme](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

Un réseau LSTM est organisé de manière similaire à un RNN, mais il y a deux états qui sont transmis de couche en couche : l'état réel $c$ et le vecteur caché $h$. À chaque unité, le vecteur caché $h_{t-1}$ est combiné avec l'entrée $x_t$, et ensemble, ils contrôlent ce qui arrive à l'état $c_t$ et à la sortie $h_{t}$ via des **portes**. Chaque porte utilise une activation sigmoïde (avec une sortie dans l'intervalle $[0,1]$), que l'on peut considérer comme un masque binaire lorsqu'elle est multipliée par le vecteur d'état. Les LSTM possèdent les portes suivantes (de gauche à droite sur l'image ci-dessus) :
* **Porte d'oubli**, qui détermine quelles composantes du vecteur $c_{t-1}$ doivent être oubliées et lesquelles doivent être conservées.
* **Porte d'entrée**, qui détermine la quantité d'informations provenant du vecteur d'entrée et du vecteur caché précédent à incorporer dans le vecteur d'état.
* **Porte de sortie**, qui prend le nouveau vecteur d'état et décide quelles de ses composantes seront utilisées pour produire le nouveau vecteur caché $h_t$.

Les composantes de l'état $c$ peuvent être considérées comme des indicateurs que l'on peut activer ou désactiver. Par exemple, lorsque nous rencontrons le nom *Alice* dans une séquence, nous supposons qu'il s'agit d'une femme et activons l'indicateur dans l'état qui signale la présence d'un nom féminin dans la phrase. Lorsque nous rencontrons ensuite les mots *et Tom*, nous activons l'indicateur signalant la présence d'un nom pluriel. Ainsi, en manipulant l'état, nous pouvons suivre les propriétés grammaticales de la phrase.

> **Note** : Voici une excellente ressource pour comprendre les mécanismes internes des LSTM : [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) par Christopher Olah.

Bien que la structure interne d'une cellule LSTM puisse sembler complexe, Keras masque cette implémentation dans la couche `LSTM`, donc la seule chose que nous devons faire dans l'exemple ci-dessus est de remplacer la couche récurrente :


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


## RNN bidirectionnels et multicouches

Dans nos exemples jusqu'à présent, les réseaux récurrents fonctionnent du début d'une séquence jusqu'à la fin. Cela nous semble naturel car cela suit la même direction que celle dans laquelle nous lisons ou écoutons un discours. Cependant, pour des scénarios nécessitant un accès aléatoire à la séquence d'entrée, il est plus logique d'exécuter le calcul récurrent dans les deux directions. Les RNN qui permettent des calculs dans les deux directions sont appelés **RNN bidirectionnels**, et ils peuvent être créés en enveloppant la couche récurrente avec une couche spéciale `Bidirectional`.

> **Note** : La couche `Bidirectional` crée deux copies de la couche qu'elle contient et définit la propriété `go_backwards` de l'une de ces copies sur `True`, ce qui lui permet de parcourir la séquence dans la direction opposée.

Les réseaux récurrents, qu'ils soient unidirectionnels ou bidirectionnels, capturent des motifs au sein d'une séquence et les stockent dans des vecteurs d'état ou les renvoient comme sortie. Comme pour les réseaux convolutionnels, nous pouvons construire une autre couche récurrente après la première pour capturer des motifs de niveau supérieur, construits à partir des motifs de niveau inférieur extraits par la première couche. Cela nous amène à la notion de **RNN multicouche**, qui consiste en deux réseaux récurrents ou plus, où la sortie de la couche précédente est transmise à la couche suivante comme entrée.

![Image montrant un RNN multicouche à mémoire longue et courte](../../../../../lessons/5-NLP/16-RNN/images/multi-layer-lstm.jpg)

*Image tirée de [cet excellent article](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) par Fernando López.*

Keras facilite la construction de ces réseaux, car il suffit d'ajouter davantage de couches récurrentes au modèle. Pour toutes les couches sauf la dernière, nous devons spécifier le paramètre `return_sequences=True`, car nous avons besoin que la couche renvoie tous les états intermédiaires, et non seulement l'état final du calcul récurrent.

Construisons un LSTM bidirectionnel à deux couches pour notre problème de classification.

> **Note** : Ce code prend encore beaucoup de temps à s'exécuter, mais il nous donne la meilleure précision que nous ayons vue jusqu'à présent. Cela vaut peut-être la peine d'attendre pour voir le résultat.


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## RNNs pour d'autres tâches

Jusqu'à présent, nous nous sommes concentrés sur l'utilisation des RNNs pour classifier des séquences de texte. Mais ils peuvent gérer bien d'autres tâches, comme la génération de texte et la traduction automatique — nous aborderons ces tâches dans la prochaine unité.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
